In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
df_fire_hasoc_ml_train = pd.read_excel("../data/malayalam hasoc/Malayalam_offensive_data_Training-YT.xlsx",index_col=0)
df_fire_hasoc_ml_train = df_fire_hasoc_ml_train.dropna()

print(df_fire_hasoc_ml_train.info())

fire_hasoc_ml_train_sentences = df_fire_hasoc_ml_train.values[:,0]
fire_hasoc_ml_train_labels = df_fire_hasoc_ml_train.values[:,1]

labels_list = list(set(fire_hasoc_ml_train_labels))

df_fire_hasoc_ml_test = pd.read_csv("../data/malayalam hasoc/malayalam_hasoc_tanglish_test_withlabels.tsv",sep="\t")
df_fire_hasoc_ml_test = df_fire_hasoc_ml_test.dropna()

print(df_fire_hasoc_ml_test.info())
fire_hasoc_ml_test_sentences = df_fire_hasoc_ml_test.values[:,-2]
fire_hasoc_ml_test_labels = df_fire_hasoc_ml_test.values[:,-1]


x_train,x_dev,y_train,y_dev = train_test_split(fire_hasoc_ml_train_sentences,fire_hasoc_ml_train_labels,test_size=0.25, random_state=42)

new_fire_hasoc_ml_test_sentences=[]

for item in fire_hasoc_ml_test_sentences:
    if "\t" not in item:
        new_fire_hasoc_ml_test_sentences.append(item)
    else:
        values = item.split("\t")
        new_fire_hasoc_ml_test_sentences.append(values[-1])
fire_hasoc_ml_test_sentences = new_fire_hasoc_ml_test_sentences

x_test = fire_hasoc_ml_test_sentences
y_test = fire_hasoc_ml_test_labels



print(len(x_train),len(x_dev),len(x_test)," total = ",len(x_train)+len(x_dev)+len(x_test))

total_sentences = []
total_sentences.extend(fire_hasoc_ml_train_sentences)
total_sentences.extend(fire_hasoc_ml_test_sentences)

total_labels = []
total_labels.extend(fire_hasoc_ml_train_labels)
total_labels.extend(fire_hasoc_ml_test_labels)

from collections import Counter
Counter(total_labels)

<class 'pandas.core.frame.DataFrame'>
Index: 3999 entries, MA_YT001 to MA_YT4000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweets  3999 non-null   object
 1   Labels  3999 non-null   object
dtypes: object(2)
memory usage: 93.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 949
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   MA_YT5000                             950 non-null    object
 1   Chenkol vendath thanne aayirunnu....  950 non-null    object
 2   NOT                                   950 non-null    object
dtypes: object(3)
memory usage: 29.7+ KB
None
2999 1000 950  total =  4949


Counter({'NOT': 2519, 'OFF': 2430})

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total_sentences)
print(X.shape)

(4949, 22682)


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_sim = cosine_similarity(X,X)

In [39]:
sum_rows_cosine_sim = np.sum(cosine_sim,axis=1)

In [40]:
dict_cosine_sim = {}
for i in range(len(y_train)+len(y_dev),sum_rows_cosine_sim.shape[0]):
    dict_cosine_sim[i] = sum_rows_cosine_sim[i]

In [41]:
sorted_dict = dict(sorted(dict_cosine_sim.items(), key=lambda item: item[1],reverse=True))

In [42]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(200, sorted_dict.items())

In [43]:
dict_key_dissmilar={}

for key,value in n_items:
    similar_values = cosine_sim[key]
    count_dissimilar_labels = 0
    count_similar_labels=0
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label == total_labels[i]:
                count_similar_labels+=1
            else:
                count_dissimilar_labels+=1
    diff = count_dissimilar_labels-count_similar_labels
    dict_key_dissmilar[key] = diff

In [44]:
sorted_key_dict = dict(sorted(dict_key_dissmilar.items(), key=lambda item: item[1],reverse=True))
print(sorted_key_dict)

{4595: 199, 4770: 133, 4651: 128, 4660: 122, 4892: 118, 4126: 109, 4946: 108, 4558: 107, 4062: 96, 4217: 94, 4155: 88, 4510: 88, 4882: 82, 4152: 81, 4079: 80, 4684: 78, 4291: 78, 4075: 76, 4183: 74, 4735: 72, 4826: 72, 4675: 71, 4580: 70, 4052: 69, 4263: 68, 4715: 67, 4457: 65, 4743: 65, 4057: 59, 4176: 49, 4586: 48, 4289: 42, 4218: 42, 4387: 34, 4275: 34, 4148: 32, 4193: 29, 4261: 24, 4276: 24, 4426: 23, 4420: 21, 4127: 21, 4645: 21, 4005: 21, 4520: 18, 4436: 15, 4225: 11, 4054: 10, 4904: 6, 4624: 6, 4451: 6, 4825: 5, 4596: 5, 4671: 4, 4598: 3, 4517: -2, 4133: -2, 4163: -9, 4543: -9, 4606: -9, 4177: -10, 4046: -12, 4080: -13, 4555: -13, 4889: -16, 4028: -17, 4271: -17, 4114: -17, 4884: -19, 4537: -23, 4599: -23, 4924: -24, 4774: -25, 4453: -25, 4898: -26, 4082: -27, 4167: -30, 4878: -30, 4061: -30, 4571: -31, 4816: -33, 4051: -34, 4565: -35, 4030: -43, 4356: -44, 4064: -46, 4347: -49, 4938: -50, 4456: -50, 4186: -50, 4696: -51, 4752: -52, 4025: -58, 4641: -60, 4917: -60, 4307: -62, 48

In [45]:
n_key_items = take(50, sorted_key_dict.items())

In [46]:
from collections import Counter
dict_key_dissmilar_label_count={}
for key,value in n_key_items:
    dissimilar_labels=[]
    similar_values = cosine_sim[key]
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label != total_labels[i]:
                dissimilar_labels.append(total_labels[i])
    counted = Counter(dissimilar_labels)
    most_common_counts = counted.most_common()
    dict_key_dissmilar_label_count[key] = most_common_counts
    
for key,value in dict_key_dissmilar_label_count.items():
    print(key,value)

4595 [('OFF', 243)]
4770 [('OFF', 205)]
4651 [('OFF', 219)]
4660 [('NOT', 442)]
4892 [('OFF', 208)]
4126 [('OFF', 273)]
4946 [('OFF', 492)]
4558 [('NOT', 445)]
4062 [('NOT', 496)]
4217 [('NOT', 376)]
4155 [('NOT', 403)]
4510 [('NOT', 388)]
4882 [('NOT', 369)]
4152 [('NOT', 408)]
4079 [('NOT', 343)]
4684 [('NOT', 515)]
4291 [('NOT', 408)]
4075 [('NOT', 259)]
4183 [('NOT', 402)]
4735 [('NOT', 551)]
4826 [('NOT', 396)]
4675 [('NOT', 409)]
4580 [('NOT', 355)]
4052 [('NOT', 243)]
4263 [('NOT', 367)]
4715 [('NOT', 337)]
4457 [('OFF', 261)]
4743 [('NOT', 386)]
4057 [('NOT', 415)]
4176 [('NOT', 353)]
4586 [('NOT', 196)]
4289 [('NOT', 182)]
4218 [('OFF', 228)]
4387 [('OFF', 296)]
4275 [('NOT', 248)]
4148 [('NOT', 353)]
4193 [('NOT', 364)]
4261 [('NOT', 523)]
4276 [('NOT', 268)]
4426 [('OFF', 244)]
4420 [('NOT', 278)]
4127 [('NOT', 207)]
4645 [('NOT', 347)]
4005 [('NOT', 245)]
4520 [('NOT', 351)]
4436 [('NOT', 345)]
4225 [('OFF', 225)]
4054 [('OFF', 214)]
4904 [('NOT', 223)]
4624 [('NOT', 284)]


In [47]:

qa_data = []
qa_labels=[]
for key,value in n_key_items:
    if value>0:
        qa_data.append(total_sentences[key])
        qa_labels.append(total_labels[key])
    
qa_df = pd.DataFrame(zip(qa_data,qa_labels),columns=['qa_data','qa_labels'])
qa_df.to_csv('malayalam_hasoc_qa.tsv',sep='\t',index=None)